In [3]:
import os
import requests
import uuid
from dotenv import load_dotenv
from typing_extensions import TypedDict, Annotated

from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command

load_dotenv()
WEATHER_API_KEY = os.getenv("Weather_api_key")
GOOGLE_API_KEY = os.getenv("google_api_key")

class ManageState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

@tool
def weather(city_name: str):
    """Fetch current weather for a given city."""
    try:
        response = requests.get(
            f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={city_name}&aqi=no"
        )
        if response.status_code == 200:
            data = response.json()
            location = data["location"]["name"]
            temp_c = data["current"]["temp_c"]
            condition = data["current"]["condition"]["text"]
            return f"The weather in {location} is {condition} with {temp_c}°C."
        else:
            return f"Error {response.status_code}: {response.content}"
    except Exception as e:
        return str(e)

Model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=GOOGLE_API_KEY)
model_with_tool = Model.bind_tools([weather])

def chatbot(state: ManageState):
    response = model_with_tool.invoke(state["messages"])
    return {"messages": [response]}

def review_node(state: ManageState):
    last_msg = state["messages"][-1]

    human_feedback = interrupt({
        "task": "Review the tool or model output.",
        "content": last_msg.content,
    })
    if human_feedback.lower()  == 'approved':
        return Command(
            update={
                "messages": [AIMessage(content=f"Human review: {human_feedback}")]
            }
        )
    else:
        return Command(
            goto="tools",
            update={
                "messages": [
                    AIMessage(content=f" User did not approve: {human_feedback}")
                ]
            }
        )
Memory = MemorySaver()
graph = StateGraph(ManageState)
graph.add_node("chatbot", chatbot)
graph.add_node("tools", ToolNode([weather]))
graph.add_node("review_node", review_node)
graph.add_edge(START, "chatbot")
graph.add_conditional_edges("chatbot", tools_condition)
graph.add_edge("tools", "review_node")
graph.add_edge("review_node", "chatbot")
workflow = graph.compile(checkpointer=Memory)


thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}



while True:
    user_prompt = input("Ask me anything (or type 'exit' to quit): ").strip()
    if user_prompt.lower() == "exit":
        print("👋 Exiting chat.")
        break

    initial_state = {"messages": [HumanMessage(content=user_prompt)]}
    interrupted = False  # Track if interruption occurred
    result = None

    for event in workflow.stream(initial_state, config=config):
        if "__interrupt__" in event:
            interrupted = True
            interrupt_data = event["__interrupt__"][0]
            print("🚨 Interrupt triggered!")
            print(interrupt_data.value)

            feedback = input("Approve or Reject? ").strip().lower()
            feedback_data = {"decision": feedback}

            # Resume the workflow with feedback
            result = workflow.resume(feedback_data, config=config)
            print("✅ After resume:", result)

    # If no interrupt happened, it means the workflow finished cleanly
    if not interrupted:
        print("✅ Workflow completed normally — no human input needed.")
        print("Result:", result)
        # Don't break here — let the user keep chatting
        continue


✅ Workflow completed normally — no human input needed.
Result: None
✅ Workflow completed normally — no human input needed.
Result: None
👋 Exiting chat.
